# Starter API Access Pipeline
---

## Imports

In [360]:
# Dependencies
# import datetime as dt
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

import havoc_helper as havoc
import api_keys

In [361]:
fema_decl = pd.read_csv('DisasterDeclarationsSummaries.csv', low_memory = False)
fema_decl.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5522-CA,5522,CA,FM,2024-07-27T00:00:00.000Z,2024,Fire,BOREL FIRE,0,0,...,99029,Kern (County),24102,NaN,2024072701,9,R,2024-08-27T18:22:14.800Z,51ec819011c936f387edf10b9d5839b35419ca95,9b73e19b-d326-4992-8da1-7e658d97607c


In [362]:
# Removing unwanted/unusable columns
fema_decl_clean = fema_decl[
    [
        'disasterNumber', 'declarationRequestNumber', 'declarationTitle', 'incidentType', 
        'declarationType', 'designatedArea', 'state', 'region', 'declarationDate', 
        'incidentBeginDate', 'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared', 
        'paProgramDeclared', 'hmProgramDeclared'
    ]
].copy()
fema_decl_clean['declarationDate'] = fema_decl_clean[
    'declarationDate'
    ].str.slice_replace(10)
fema_decl_clean['incidentBeginDate'] = fema_decl_clean[
    'incidentBeginDate'
    ].str.slice_replace(10)
display(fema_decl_clean)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67351,9,53010,FLOOD,Flood,DR,Statewide,TX,6,1953-06-19,1953-06-19,0,0,1,1,1
67352,8,53008,FLOOD,Flood,DR,Statewide,IA,7,1953-06-11,1953-06-11,0,0,1,1,1
67353,7,53009,TORNADO,Tornado,DR,Statewide,MA,1,1953-06-11,1953-06-11,0,0,1,1,1
67354,2,53003,TORNADO & HEAVY RAINFALL,Tornado,DR,Statewide,TX,6,1953-05-15,1953-05-15,0,0,1,1,1


In [363]:
# Unique FEMA Disaster Types
disaster_types = fema_decl_clean['incidentType'].unique()
pprint(disaster_types)

array(['Fire', 'Severe Storm', 'Straight-Line Winds', 'Flood',
       'Winter Storm', 'Hurricane', 'Tornado', 'Mud/Landslide',
       'Tropical Storm', 'Snowstorm', 'Earthquake', 'Coastal Storm',
       'Other', 'Severe Ice Storm', 'Biological', 'Dam/Levee Break',
       'Typhoon', 'Volcanic Eruption', 'Freezing', 'Toxic Substances',
       'Chemical', 'Terrorist', 'Drought', 'Human Cause',
       'Fishing Losses', 'Tsunami'], dtype=object)


In [364]:
# TODO -> Remove volcanoes, earthquake, mud/landslide
disaster_natural = pd.DataFrame(
    {
        'incidentType': [
            'Fire', 'Severe Storm', 'Straight-Line Winds', 
            'Flood', 'Winter Storm', 'Hurricane', 'Tornado',
            'Mud/Landslide', 'Tropical Storm', 'Snowstorm', 
            'Earthquake', 'Coastal Storm', 'Severe Ice Storm', 
            'Typhoon', 'Volcanic Eruption', 'Freezing', 
            'Drought', 'Tsunami'
        ]
    }
)

In [365]:
# Filter for just natural disaster types
fema_disasters = pd.merge(fema_decl_clean, disaster_natural, how = 'right', on = 'incidentType')

# TODO: Do we want renamed columns? Or do we like the current columns?
# fema_disasters = fema_disasters.rename(
#     columns = {
#         'disasterNumber': 'disaster_number', 'designatedArea': 'area',
#         'declarationRequestNumber': 'request_number', 'declarationTitle': 'title',
#         'incidentType': 'disaster_type'

#     }
# )
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [366]:
# Counts of declaration types 
# DR = Major Disasters, EM = Emergency Declaratation, FM = Fire Management)
# For more info see README (in progress)
fema_disasters['declarationType'].value_counts()

declarationType
DR    41116
EM    15991
FM     1994
Name: count, dtype: int64

In [367]:
# Counts of natural disaster types
fema_disasters['incidentType'].value_counts()

incidentType
Severe Storm           18399
Hurricane              13721
Flood                  11093
Fire                    3759
Snowstorm               3707
Severe Ice Storm        2942
Tornado                 1623
Drought                 1292
Tropical Storm          1047
Coastal Storm            637
Freezing                 301
Earthquake               228
Typhoon                  130
Winter Storm             117
Volcanic Eruption         51
Mud/Landslide             43
Tsunami                    9
Straight-Line Winds        2
Name: count, dtype: int64

In [368]:
# Validating no NaNs
fema_disasters.count()

disasterNumber              59101
declarationRequestNumber    59101
declarationTitle            59101
incidentType                59101
declarationType             59101
designatedArea              59101
state                       59101
region                      59101
declarationDate             59101
incidentBeginDate           59101
tribalRequest               59101
ihProgramDeclared           59101
iaProgramDeclared           59101
paProgramDeclared           59101
hmProgramDeclared           59101
dtype: int64

In [369]:
# Filter: Natural disasters per State
fema_disasters['state'].value_counts()

state
TX    4779
FL    2635
KY    2554
MO    2534
LA    2398
GA    2333
VA    2308
OK    2305
NC    2110
PR    1947
MS    1777
IA    1719
KS    1607
AL    1603
CA    1557
TN    1532
AR    1472
MN    1406
NE    1336
ND    1280
IN    1279
NY    1278
SD    1181
WV    1148
PA    1118
IL    1109
OH    1102
SC    1039
WA     902
WI     725
MI     595
NJ     556
OR     536
CO     531
MT     507
ME     426
MD     399
NM     380
VT     377
MA     365
NH     300
ID     276
AZ     271
CT     245
AK     211
NV     198
UT     183
RI     106
HI      97
WY      86
VI      79
MP      67
AS      66
MH      53
DE      47
FM      31
GU      20
DC      19
PW       1
Name: count, dtype: int64

In [370]:
# Filter: Natural disasters per hit area
fema_disasters['designatedArea'].value_counts()

designatedArea
Washington (County)                          510
Jefferson (County)                           462
Statewide                                    450
Jackson (County)                             450
Franklin (County)                            441
                                            ... 
Southern Ute Indian Reservation                1
Ute Mountain Indian Reservation (Also NM)      1
Nondalton (ANV/ANVSA)                          1
Salt River Pima-Maricopa Indian Community      1
Namorik (County-equivalent)                    1
Name: count, Length: 2201, dtype: int64

## Geoapify Geocoding API

In [371]:
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [372]:
# Splitting up area from the areaType in designatedArea
geocode_df = fema_disasters.copy()
geocode_df[['area', 'areaType']] = geocode_df['designatedArea'].str.split('(', n = 1, expand = True)
areas = geocode_df.pop('area')
areaTypes = geocode_df.pop('areaType')
areaTypes = '(' + areaTypes
geocode_df.insert(5, 'area', areas)
geocode_df.insert(6, 'areaType', areaTypes)
del geocode_df['designatedArea']

In [373]:
# Regex for cleaning up text in areaType
geocode_df['areaType'] = geocode_df['areaType'].str.extract(r'\(([^)]+)\)')
geocode_df['areaType'] = geocode_df['areaType'].str.replace(r'\b(of|Also|and)\b', '', regex = True).str.strip()
display(geocode_df)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,area,areaType,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe,County,NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington,County,OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson,County,OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes,County,OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern,County,CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu,County,HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui,County,HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos,County,OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry,County,OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [374]:
geocode_df['areaType'].unique()

array(['County', nan, 'Parish', 'Borough', 'Census Area',
       'Indian Reservation', 'Municipio', 'ANV/ANVSA', 'NM',
       'County-equivalent', 'OTSA', 'NM  UT', 'Sisseton', 'Reservation',
       'District', 'Island', 'AZ  UT', 'Township',
       'Unorganized Territory', 'Police Jury Ward', 'Nett Lake', 'TDSA',
       'KS', 'Municipality', 'CCD', 'City  Borough',
       'Native Regional Corporation', 'State', 'Census Subarea'],
      dtype=object)

In [375]:
# Setting up the Data Frame containing Area Types with 1
# or more states instead of classic 'County' or 'Reservation'
# --> Result is duplicated incidents but for the additional states

# Multi-state regex location in 'areaType' column
multiState_regex = r'\b[A-Z]{2}(?:\s[A-Z]{2})*\b'
states_to_split = geocode_df['areaType'].str.contains(multiState_regex, na = False)
new_rows = []

# Iterate through just the Area Types that need to be split and duplicated
for index, row in geocode_df[states_to_split].iterrows():
    states = row['areaType'].split()
    for state in states:
        new_row = row.copy()
        new_row['state'] = state
        new_rows.append(new_row)

geocode_df = pd.concat(     # Concatenate duplicated rows for multiple states to the main dataframe
    [geocode_df, pd.DataFrame(new_rows)], ignore_index = True
    ).reset_index(drop = True).copy()

# Set Non-Capture Regex and location rows that can have areaType set to 'Reservation'
reservation_regex = r'(?:Indian)|(?:Reservation)|(?:Tribe)|(?:Trust Lands)|(?:TDSA)'
geocode_df.loc[geocode_df['area'].str.contains(reservation_regex, case = False, regex = True), 'areaType'] = 'Reservation'

In [376]:
# Nitty-Gritty edits (less than 1000 rows each)
# Null `areaTypes`: Regional Educational Attendence Area, Crow/Northern Cheyenne Area
null_areaType = geocode_df['areaType'].isnull() # Set null `areaType` variable for easy-access

# Change Puerto Rican spelling of Municipality
geocode_df.loc[geocode_df['areaType'] == 'Municipio', 'areaType'] = 'Municipality'

# Found under Null types (NaN)
# Location Statewide listed areas, copying abbreviations from `state` column
# to the `area` column, extending them to full length names and then setting 
# the `areaType` to State
statewide_locs = (geocode_df['area'] == 'Statewide')
geocode_df.loc[statewide_locs, 'areaType'] = 'State'
geocode_df.loc[statewide_locs, 'area'] = geocode_df.loc[statewide_locs, 'state']
geocode_df['area'] = geocode_df['area'].replace(havoc.state_dict)

# Found under Null types (NaN) 
# Less than 20 entries of 'Carson City' -> Correct `areaType` to `City`
geocode_df.loc[geocode_df['area'] == 'Carson City', 'areaType'] = 'City'

# Using geocode_df['areaType'].unique() location type 'City  Borough'
# Only 1 entry of Wranell, AK (Alaska) -> Correct to 'Borough'
geocode_df.loc[geocode_df['areaType'] == 'City  Borough', 'areaType'] = 'Borough'

# Regional Educational Attendance Area `area` with Null `areaType` (it's the same with or without null spec)
# TODO: Correct to `areaType` -> 'Regional Educational Attendance Area' or 'School District'
# geocode_df.loc[reaa, 'area']
reaa_regex = r'Regional Educational Att'
reaa_loc = geocode_df['area'].str.contains(reaa_regex, case = False, regex = True)
geocode_df.loc[reaa_loc, 'areaType'] = 'School District'
geocode_df.loc[reaa_loc, 'area'] = geocode_df.loc[reaa_loc, 'area'].str.replace(reaa_regex, '', regex = True)

# Set `areaType` = 'Village' when `area` contains the word 'Pueblo'
pueblo_regex = r'(?:Pueblo)'
pueblo_loc = geocode_df['area'].str.contains(pueblo_regex, case = False, regex = True)
geocode_df.loc[pueblo_loc, 'areaType'] = 'Village'

# LAST STEP:
# Null areaType with States in list to compare by
# LARGE NULL GROUP: `areaType` is NaN and `state` is 'VA' 537 Rows
# Correct `areaType` -> 'City'
states_toCheck = ['IA', 'MD', 'MN', 'MO', 'VA']
geocode_df.loc[((null_areaType) & (
    (geocode_df['state'].isin(states_toCheck)))),
    'areaType'] = 'City'

country_toCheck = ['FM', 'MH']
geocode_df.loc[((null_areaType) & (
    (geocode_df['state'].isin(country_toCheck)))),
    'areaType'] = 'Country'

In [377]:
geocode_df.iloc[812, 5].split()

['Crow/Northern', 'Cheyenne', 'Area']

In [378]:
# Examining what is still Null and tackling each null type as it's revealed
geocode_df.loc[geocode_df['areaType'].isnull(), :].tail(45)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,area,areaType,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
812,4074,12061,WILDFIRE,Fire,DR,Crow/Northern Cheyenne Area,NaN,MT,8,2012-08-02,2012-06-25,0,0,0,1,1
22380,4730,23085,FLOODING,Flood,DR,Kusilvak Census Area,NaN,AK,10,2023-08-23,2023-05-12,0,1,0,0,1


In [379]:
[print(x) for x in geocode_df[pueblo_loc, 'area']]

InvalidIndexError: (0        False
1        False
2        False
3        False
4        False
         ...  
59122    False
59123    False
59124    False
59125    False
59126    False
Name: area, Length: 59127, dtype: bool, 'area')

In [ ]:
geocode_df.loc[geocode_df['areaType'] == 'ANV/ANVSA']

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,area,areaType,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
2169,2551,4096,AK-TAYLOR COMPLEX FIRE-09-01-2004,Fire,FM,Dot Lake,ANV/ANVSA,AK,10,2004-09-01,2004-08-09,0,0,0,1,0
2170,2551,4096,AK-TAYLOR COMPLEX FIRE-09-01-2004,Fire,FM,Northway,ANV/ANVSA,AK,10,2004-09-01,2004-08-09,0,0,0,1,0
2171,2551,4096,AK-TAYLOR COMPLEX FIRE-09-01-2004,Fire,FM,Tanacross,ANV/ANVSA,AK,10,2004-09-01,2004-08-09,0,0,0,1,0
2172,2551,4096,AK-TAYLOR COMPLEX FIRE-09-01-2004,Fire,FM,Tetlin,ANV/ANVSA,AK,10,2004-09-01,2004-08-09,0,0,0,1,0
2173,2551,4096,AK-TAYLOR COMPLEX FIRE-09-01-2004,Fire,FM,Tok,ANV/ANVSA,AK,10,2004-09-01,2004-08-09,0,0,0,1,0
2414,2365,1074,AK - RED FOX FIRE - 06/30/01,Fire,FM,Tok,ANV/ANVSA,AK,10,2001-06-30,2001-06-29,0,0,0,1,0
3722,813,88008,FIRE,Fire,DR,Tanana,ANV/ANVSA,AK,10,1988-03-11,1988-02-10,0,0,0,1,0
3737,493,76029,FIRE & FREEZING,Fire,DR,Tanana,ANV/ANVSA,AK,10,1975-12-18,1975-12-18,0,0,1,1,0
9356,4853,24196,SEVERE STORM AND FLOODING,Severe Storm,DR,Kipnuk,ANV/ANVSA,AK,10,2025-01-01,2024-08-16,1,0,0,1,1
16624,1445,2150,"SEVERE WINTER STORMS, FLOODING, COASTAL EROSIO...",Severe Storm,DR,Chignik Lagoon,ANV/ANVSA,AK,10,2002-12-04,2002-10-23,0,1,1,1,1


In [332]:
# geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'
# geoapify_params = {
#     'apiKey': api_keys.geoapify_key
#     }

# for index, row in fema_disasters.iterrows():
#     geoapify_params['state'] = row['state']
#     geocode_response = requests.get(geoapify_base, geoapify_params).json()
#     pprint(geocode_response)
#     break